In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from arch.univariate import GARCH, ConstantMean, Normal

sns.set_style("darkgrid")
plt.rc("figure", figsize=(16, 6))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

In [2]:
import arch.data.nasdaq
from arch import arch_model
data = arch.data.nasdaq.load()
nasdaq = data["Adj Close"]
print(nasdaq.head())

Date
1999-01-04    2208.050049
1999-01-05    2251.270020
1999-01-06    2320.860107
1999-01-07    2326.090088
1999-01-08    2344.409912
Name: Adj Close, dtype: float64


<Axes: xlabel='Date'>

In [15]:
mod = arch_model(rets, mean='constant', p=1, o=1, q=1)
res = mod.fit(disp="off")
res

                   Constant Mean - GJR-GARCH Model Results                    
Dep. Variable:              Adj Close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                  GJR-GARCH   Log-Likelihood:               -8196.75
Distribution:                  Normal   AIC:                           16403.5
Method:            Maximum Likelihood   BIC:                           16436.1
                                        No. Observations:                 5030
Date:                Wed, Feb 26 2025   Df Residuals:                     5029
Time:                        00:00:36   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0376  1.476e-02      2.549  1.081e-02 [8.

In [25]:
class ScenarioBootstrapRNG(object):
    def __init__(self, shocks, random_state):
        self._shocks = np.asarray(shocks)  # 1d
        self._rs = random_state
        self.n = shocks.shape[0]

    def rng(self, size):
        idx = self._rs.randint(0, self.n, size=size)
        return self._shocks[idx]


random_state = np.random.RandomState(1)
std_shocks = res.resid / res.conditional_volatility
shocks = std_shocks["2008-08-01":"2008-11-10"]
scenario_bootstrap = ScenarioBootstrapRNG(shocks, random_state)
bs_forecasts = res.forecast(
    start="1-1-2017", method="simulation", horizon=10, rng=scenario_bootstrap.rng
)
print(bs_forecasts.residual_variance.dropna().head())

                h.01      h.02      h.03      h.04      h.05      h.06  \
Date                                                                     
2017-01-03  0.623295  0.676081  0.734322  0.779325  0.828189  0.898202   
2017-01-04  0.599455  0.645237  0.697133  0.750169  0.816280  0.888417   
2017-01-05  0.567297  0.610493  0.665995  0.722954  0.777860  0.840369   
2017-01-06  0.542506  0.597387  0.644534  0.691387  0.741206  0.783319   
2017-01-09  0.515452  0.561312  0.611026  0.647824  0.700559  0.757398   

                h.07      h.08      h.09      h.10  
Date                                                
2017-01-03  0.958215  1.043704  1.124684  1.203893  
2017-01-04  0.945120  1.013400  1.084042  1.158148  
2017-01-05  0.889032  0.961424  1.022412  1.097192  
2017-01-06  0.840667  0.895559  0.957266  1.019497  
2017-01-09  0.820788  0.887791  0.938708  1.028614  


In [26]:
forecasts = res.forecast(start="1-1-2017", horizon=10)
print(forecasts.residual_variance.dropna().head())

                h.01      h.02      h.03      h.04      h.05      h.06  \
Date                                                                     
2017-01-03  0.623295  0.637504  0.651549  0.665431  0.679154  0.692717   
2017-01-04  0.599455  0.613940  0.628257  0.642408  0.656397  0.670223   
2017-01-05  0.567297  0.582153  0.596837  0.611352  0.625699  0.639880   
2017-01-06  0.542506  0.557649  0.572616  0.587410  0.602034  0.616488   
2017-01-09  0.515452  0.530906  0.546183  0.561282  0.576208  0.590961   

                h.07      h.08      h.09      h.10  
Date                                                
2017-01-03  0.706124  0.719376  0.732475  0.745423  
2017-01-04  0.683890  0.697399  0.710752  0.723950  
2017-01-05  0.653897  0.667753  0.681448  0.694985  
2017-01-06  0.630776  0.644899  0.658858  0.672656  
2017-01-09  0.605543  0.619957  0.634205  0.648288  


In [29]:
sim_forecasts = res.forecast(start="1-1-2017", method="simulation", horizon=10)
print(sim_forecasts.residual_variance.dropna().head())

                h.01      h.02      h.03      h.04      h.05      h.06  \
Date                                                                     
2017-01-03  0.623295  0.632241  0.643776  0.656672  0.666954  0.685788   
2017-01-04  0.599455  0.614732  0.627833  0.641635  0.660119  0.673133   
2017-01-05  0.567297  0.580060  0.596387  0.611111  0.625573  0.642829   
2017-01-06  0.542506  0.558387  0.571548  0.586721  0.597646  0.609741   
2017-01-09  0.515452  0.526772  0.538204  0.554115  0.572644  0.586008   

                h.07      h.08      h.09      h.10  
Date                                                
2017-01-03  0.692380  0.704271  0.720579  0.731049  
2017-01-04  0.691355  0.703867  0.715444  0.720587  
2017-01-05  0.657339  0.669877  0.680275  0.693756  
2017-01-06  0.616056  0.632923  0.650148  0.660345  
2017-01-09  0.597205  0.616717  0.631336  0.646467  


In [30]:
df = pd.concat(
    [
        forecasts.residual_variance.iloc[-1],
        sim_forecasts.residual_variance.iloc[-1],
        scenario_forecasts.residual_variance.iloc[-1],
        bs_forecasts.residual_variance.iloc[-1],
    ],
    axis=1,
)
df.columns = ["Analytic", "Simulation", "Scenario Sim", "Bootstrp Scenario"]
# Plot annualized vol
subplot = np.sqrt(252 * df).plot(legend=False)
legend = subplot.legend(frameon=False)


NameError: name 'scenario_forecasts' is not defined

In [28]:
fig, axes = plt.subplots(1, 2)
colors = sns.color_palette("dark")
# The paths for the final observation
sim_paths = sim_forecasts.simulations.residual_variances[-1].T
bs_paths = bs_forecasts.simulations.residual_variances[-1].T

x = np.arange(1, 11)
# Plot the paths and the mean, set the axis to have the same limit
axes[0].plot(x, np.sqrt(252 * sim_paths), color=colors[1], alpha=0.05)
axes[0].plot(
    x, np.sqrt(252 * sim_forecasts.residual_variance.iloc[-1]), color="k", alpha=1
)
axes[0].set_title("Model-based Simulation")
axes[0].set_xticks(np.arange(1, 11))
axes[0].set_xlim(1, 10)
axes[0].set_ylim(20, 100)

axes[1].plot(x, np.sqrt(252 * bs_paths), color=colors[2], alpha=0.05)
axes[1].plot(
    x, np.sqrt(252 * bs_forecasts.residual_variance.iloc[-1]), color="k", alpha=1
)
axes[1].set_xticks(np.arange(1, 11))
axes[1].set_xlim(1, 10)
axes[1].set_ylim(20, 100)
title = axes[1].set_title("Bootstrap Scenario")

NameError: name 'sim_forecasts' is not defined